In [ ]:
#collect paddlex imblearn
!pip install paddlex==2.0.0rc4 -i https://mirror.baidu.com/pypi/simple
!pip install imblearn

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 6.8MB 12.6MB/s eta 0:00:01
ERROR: imbalanced-learn 0.8.0 has requirement scikit-learn>=0.24, but you'll have scikit-learn 0.23.2 which is incompatible.
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.20.3 which is incompatible.
  Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 22.3MB 8.7MB/s eta 0:00:011
ERROR: paddlex 2.0.0rc4 has requirement scikit-learn==0.23.2, but you'll have scikit-learn 0.24.2 which is incompatible.
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.20.3 which is incompatible.
  Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


In [ ]:
# #unzip data
# !unzip data/data105081/test.zip -d work
# !unzip data/data105081/train.zip -d work

Archive:  data/data105081/test.zip
replace work/test/06796.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  data/data105081/train.zip
replace work/train/happy/im4403.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')                   # if want to use the default style, set 'classic'
plt.rcParams['ytick.right']     = True
plt.rcParams['ytick.labelright']= True
plt.rcParams['ytick.left']      = False
plt.rcParams['ytick.labelleft'] = False
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx
from paddlex import transforms as T

In [ ]:
#split dataset
!paddlex --split_dataset --format ImageNet --dataset_dir work/train --val_value 0.2 

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."
[08-20 21:10:43 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format ImageNet --dataset_dir work/train --val_value 0.2
[08-20 21:10:43 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_cont

In [ ]:
# data augument
train_transforms = T.Compose([
                            T.RandomBlur(), 
                            T.RandomHorizontalFlip(), 
                            T.Normalize(),
                            ])                        
eval_transforms = T.Compose([T.Normalize()])

In [34]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='work/train',
    file_list='work/train/train_list.txt',
    label_list='work/train/labels.txt',
    transforms=train_transforms,
    shuffle=True)
    
eval_dataset = pdx.datasets.ImageNet(
    data_dir='work/train',
    file_list='work/train/val_list.txt',
    label_list='work/train/labels.txt',
    transforms=eval_transforms)

2021-08-20 21:35:54 [INFO]	Starting to read file list from dataset...
2021-08-20 21:35:55 [INFO]	22968 samples in file work/train/train_list.txt
2021-08-20 21:35:55 [INFO]	Starting to read file list from dataset...
2021-08-20 21:35:55 [INFO]	5741 samples in file work/train/val_list.txt


In [44]:
# #smote imbalance todo
# from imblearn.over_sampling import SMOTE 
# X=train_dataset.file_list
# print(X[0]['image'])
# Y=train_dataset.file_list
# sm = SMOTE(random_state=1) 


work/train/angry/im2123.png


In [ ]:
num_classes = len(train_dataset.labels)
model = pdx.cls.ResNet101_vd_ssld(num_classes=num_classes)
model.train(num_epochs=10,
            train_dataset=train_dataset,
            train_batch_size=128,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[4, 6, 8],
            save_interval_epochs=1,
            learning_rate=0.025,
            save_dir='output/ResNet101_vd_ssld',
            use_vdl=True)

2021-08-20 21:21:45 [INFO]	Downloading ResNet101_vd_ssld_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ResNet101_vd_ssld_pretrained.pdparams


100%|██████████| 263250/263250 [00:04<00:00, 58569.87KB/s]


2021-08-20 21:21:49 [INFO]	Loading pretrained model from output/ResNet101_vd_ssld/pretrain/ResNet101_vd_ssld_pretrained.pdparams
2021-08-20 21:21:51 [WARNING]	[SKIP] Shape of pretrained params out.weight doesn't match.(Pretrained: [2048, 1000], Actual: [2048, 7])
2021-08-20 21:21:51 [WARNING]	[SKIP] Shape of pretrained params out.bias doesn't match.(Pretrained: [1000], Actual: [7])
2021-08-20 21:21:51 [INFO]	There are 530/532 variables loaded into ResNet101_vd_ssld.
2021-08-20 21:21:53 [INFO]	[TRAIN] Epoch=1/10, Step=10/179, loss=1.988329, acc1=0.164062, acc5=0.890625, lr=0.025000, time_each_step=0.17s, eta=0:5:18
2021-08-20 21:21:54 [INFO]	[TRAIN] Epoch=1/10, Step=20/179, loss=1.829863, acc1=0.351562, acc5=0.921875, lr=0.025000, time_each_step=0.12s, eta=0:3:32
2021-08-20 21:21:55 [INFO]	[TRAIN] Epoch=1/10, Step=30/179, loss=1.583818, acc1=0.359375, acc5=0.953125, lr=0.025000, time_each_step=0.12s, eta=0:3:30
2021-08-20 21:21:57 [INFO]	[TRAIN] Epoch=1/10, Step=40/179, loss=1.641431, a

In [32]:
test=pd.read_csv('work/sample_submit.csv')
model = pdx.load_model('output/ResNet101_vd_ssld/best_model/')
labels=[]
for index, item in test.iterrows():  
    image_name = os.path.join('work/test', item['name'])
    label = model.predict(image_name)
    labels.append(label)
print("Predict Done:", len(labels))

2021-08-20 21:31:02 [INFO]	Model[ResNet101_vd_ssld] loaded.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


KeyboardInterrupt: 

In [ ]:
test['label']=labels
for i in range(len(labels)):
    test['label'][i]=test['label'][i][0]['category']

# 不要index
test.to_csv('result.csv', index=False)